# Build an Agent
Adapted from [Build an Agent](https://python.langchain.com/v0.2/docs/tutorials/agents/)

By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents. Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs to pass them. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish.

In this tutorial we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.

## Concepts


In following this tutorial, you will learn how to:

* Use [language models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), in particular their tool calling ability
* Use a Search [Tool](https://python.langchain.com/v0.2/docs/concepts/#tools) to look up information from the Internet
* Compose a [LangGraph Agent](https://python.langchain.com/v0.2/docs/concepts/#agents), which use an LLM to determine actions and then execute them
* Debug and trace your application using [LangSmith](https://python.langchain.com/v0.2/docs/concepts/#langsmith)

## End-to-end agent
The code snippet below represents a fully functional agent that uses an LLM to decide which tools to use. It is equipped with a generic search tool. It has conversational memory - meaning that it can be used as a multi-turn chatbot.

In the rest of the guide, we will walk through the individual components and what each part does - but if you want to just grab some code and get started, feel free to use this!
```python
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import chat_agent_executor

# Create the agent
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = chat_agent_executor.create_tool_calling_executor(
    model, tools, checkpointer=memory
)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")
```

* API Reference: [ChatAnthropic](https://api.python.langchain.com/en/latest/chat_models/langchain_anthropic.chat_models.ChatAnthropic.html) | [TavilySearchResults](https://api.python.langchain.com/en/latest/tools/langchain_community.tools.tavily_search.tool.TavilySearchResults.html) | [HumanMessage](https://api.python.langchain.com/en/latest/messages/langchain_core.messages.human.HumanMessage.html)

## Setup
We will need to multiple credentials to connect to Langchain API, OpenAI API and, Tavily API

### Tavily
We will be using [Tavily (a search engine)](https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/) as a tool. In order to use it, you will need to get and set an API key:
```bash
export TAVILY_API_KEY="..."
```

### Setting credentials with python-dot-env
Load credentials from a .env file and the [python-dotenv package](https://pypi.org/project/python-dotenv/)
```toml
LANGCHAIN_API_KEY="<KEY>"
OPENAI_API_KEY="<KEY>"
TAVILY_API_KEY="<KEY>"
```

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with [LangSmith](https://smith.langchain.com/).

After you sign up at the link above, make sure to set your environment variables to start logging traces:
```bash
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

In [5]:
import os
from dotenv import load_dotenv

os.environ["LANGCHAIN_TRACING_V2"] = "true"

load_dotenv()
assert os.environ["LANGCHAIN_API_KEY"]
assert os.environ["OPENAI_API_KEY"]
assert os.environ["TAVILY_API_KEY"]

## Define tools
We first need to create the tools we want to use. Our main tool of choice will be Tavily - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

* API Reference: [TavilySearchResults](https://api.python.langchain.com/en/latest/tools/langchain_community.tools.tavily_search.tool.TavilySearchResults.html)

In [8]:
search = TavilySearchResults(max_results=2)

search.invoke("what is the weather in Singapore")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Singapore', 'region': '', 'country': 'Singapore', 'lat': 1.29, 'lon': 103.86, 'tz_id': 'Asia/Singapore', 'localtime_epoch': 1717225825, 'localtime': '2024-06-01 15:10'}, 'current': {'last_updated_epoch': 1717225200, 'last_updated': '2024-06-01 15:00', 'temp_c': 27.0, 'temp_f': 80.6, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 8.1, 'wind_kph': 13.0, 'wind_degree': 190, 'wind_dir': 'S', 'pressure_mb': 1011.0, 'pressure_in': 29.85, 'precip_mm': 0.07, 'precip_in': 0.0, 'humidity': 89, 'cloud': 75, 'feelslike_c': 30.4, 'feelslike_f': 86.7, 'windchill_c': 27.6, 'windchill_f': 81.6, 'heatindex_c': 31.5, 'heatindex_f': 88.7, 'dewpoint_c': 23.9, 'dewpoint_f': 75.1, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 6.0, 'gust_mph': 20.4, 'gust_kph': 32.9}}"},
 {'url': 'https://www.wunderground.com/hourly/sg/singapore/ISINGA72/date/2024-06

If we want, we can create other tools. Once we have all the tools we want, we can put them in a list that we will reference later.

In [9]:
tools = [search]

# Using Language Models
We will use the OpenAI gpt-4 model to call tools

In [10]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

You can call the language model by passing in a list of messages. By default, the response is a `content` string.

In [11]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

We can now see what it is like to enable this model to do tool calling. In order to enable that we use `.bind_tools` to give the language model knowledge of these tools

In [13]:
model_with_tools = model \
                .bind_tools(tools) # The LLM has access to the tools list

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the `content` field as well as the `tool_calls` field.

In [14]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


Now, let's try calling it with some input that would expect a tool to be called.

In [17]:
response = model_with_tools.invoke(
    [HumanMessage(content="What's the weather in Singapore?")]
)

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Singapore'}, 'id': 'call_7rUHWygmHZDvi5jSiHCtW4t9'}]


We can see that there's now no content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually calll it, we'll want to create our agent.

## Create the agent
Now that we have defined the tools and the LLM, we can create the agent. We will be using LangGraph to construct the agent. Currently we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.

Now, we can initalize the agent with the LLM and the tools.

Note that we are passing in the `model`, not `model_with_tools`. That is because `create_tool_calling_executor` will call `.bind_tools` for us under the hood.

In [20]:
from langgraph.prebuilt import chat_agent_executor

agent_executor = chat_agent_executor \
    .create_tool_calling_executor(model, tools)

## Run the agent
We can now run the agent on a few queries! Note that for now, these are all __stateless__ queries (it won't remember previous interactions). Note that the agent will return the __final__ state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

First up, let's see how it responds when there's no need to call a tool:

In [21]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='4966e44c-287b-4284-9f23-1d25f367450f'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6e367fc0-3d33-433b-8823-52285b53f105-0')]

In order to see exactly what is happening under the hood (and to make sure it's not calling a tool) we can take a look at the [LangSmith trace](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)

Let's now try it out on an example where it should be invoking the tool

In [22]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in Singapore?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in Singapore?', id='da8ebb97-e259-41e6-8670-1a4732fbc0ea'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9dsIsTesBOIvnZeOi0BUJg1t', 'function': {'arguments': '{\n  "query": "current weather in Singapore"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 88, 'total_tokens': 110}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2ba81b2a-fdad-4e14-8f2d-947129629065-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Singapore'}, 'id': 'call_9dsIsTesBOIvnZeOi0BUJg1t'}]),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Singapore\', \'region\': \'\', \'country\': \'Singapore\', \'lat\': 1.29, \'lon\': 103.86, \'tz_id\': \'Asia/Singapore\', \'localtime_epoch\': 1717227574, \'localti

We can check out the [LangSmith trace](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r) to make sure it's calling the search tool effectively.

## Streaming Messages
We've seen how the agent can be called with `.invoke` to get back a final response. If the agent is executing multiple steps, that may take a while. In order to show intermediate progress, we can stream back messages as they occur.

In [25]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Singapore?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AvY2IuVt8a4QBHCGiIRyjGL7', 'function': {'arguments': '{\n  "query": "current weather in Singapore"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 88, 'total_tokens': 110}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d4ff6d32-acf4-4deb-94ac-cec123d1686c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Singapore'}, 'id': 'call_AvY2IuVt8a4QBHCGiIRyjGL7'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Singapore\', \'region\': \'\', \'country\': \'Singapore\', \'lat\': 1.29, \'lon\': 103.86, \'tz_id\': \'Asia/Singapore\', \'localtime_epoch\': 1717227574, \'localtime\': \'2024-06-01 15:39\'}, \'current\': {\'las

## Streaming tokens
In addition to streaming back messages, it is also useful to be streaming back tokens. We can do this with the `.astream_events` method.

*INFO: This .astream_events method only works with Python 3.11 or higher.*

In [26]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in Singapore?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

/home/ubuntu/.local/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in Singapore'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Singapore', 'region': '', 'country': 'Singapore', 'lat': 1.29, 'lon': 103.86, 'tz_id': 'Asia/Singapore', 'localtime_epoch': 1717227761, 'localtime': '2024-06-01 15:42'}, 'current': {'last_updated_epoch': 1717227000, 'last_updated': '2024-06-01 15:30', 'temp_c': 28.0, 'temp_f': 82.4, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 8.1, 'wind_kph': 13.0, 'wind_degree': 200, 'wind_dir': 'SSW', 'pressure_mb': 1010.0, 'pressure_in': 29.83, 'precip_mm': 0.07, 'precip_in': 0.0, 'humidity': 84, 'cloud': 75, 'feelslike_c': 32.4, 'feelslike_f': 90.4, 'windchill_c': 27.6, 'windchill_f': 81.6, 'heatindex_c': 31.5, 'heatindex_f': 88.7, 'dewpoint_c': 23.9, 'dewpoint_f': 75.1, 'vis_km': 

## Adding in memory
As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a `thread_id` when invoking the agent (so it knows which thread/conversation to resume from).

In [28]:
### Create an in-memory checkpointer using Sqllite
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

### Create an agent by passing in model, tools, and checkpointer
agent_executor = chat_agent_executor.create_tool_calling_executor(
    model=model,
    tools=tools,
    checkpointer=memory
)

config = {"configurable": 
          {"thread_id": "abc123"} # thread_id keeps track of which thread/conversation to resume from
         }

### Send prompt to agent_executor and stream output
for chunk in agent_executor.stream(
    input={"messages": [HumanMessage(content="hi im bob!")]},
    config=config,
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 85, 'total_tokens': 96}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-812fabb3-3d32-4c56-91c1-8b088bfed646-0')]}}
----


In [29]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 108, 'total_tokens': 114}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fe6ce6d7-69ae-4139-8cb3-d24a6bde6d46-0')]}}
----


If I want to start a new conversation, all I have to do is change the `thread_id` used

## Conclusion
That's a wrap! In this quick start we covered how to create a simple agent. We've then shown how to stream back a response - not only the intermediate steps, but also tokens! We've also added in memory so you can have a conversation with them. Agents are a complex topic, and there's lot to learn!

For more information on Agents, please check out the [LangGraph](https://python.langchain.com/v0.2/docs/concepts/#langgraph) documentation. This has it's own set of concepts, tutorials, and how-to guides.